In [1]:
import Pkg
Pkg.add("BSON")
Pkg.add("Flux")
Pkg.add("Statistics")
Pkg.add("Printf")






  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed CuArrays ─ v1.7.2
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [2]:
# If we don't do this, then flux won't grok GPUs
# Pinning to 1.3 was suggested in https://github.com/FluxML/Flux.jl/issues/918
Pkg.add(Pkg.PackageSpec(;name="CuArrays", version="1.3"))
# Pkg.pin("CuArrays", v"1.3")
# Pkg.add("CuArrays")
using CuArrays

 Resolving package versions...


Pkg.Types.ResolverError: Unsatisfiable requirements detected for package CuArrays [3a865a2d]:
 CuArrays [3a865a2d] log:
 ├─possible versions are: [0.2.1, 0.3.0, 0.4.0, 0.5.0, 0.6.0-0.6.2, 0.7.0-0.7.3, 0.8.0-0.8.1, 0.9.0-0.9.1, 1.0.0-1.0.2, 1.1.0, 1.2.0-1.2.1, 1.3.0, 1.4.0-1.4.7, 1.5.0, 1.6.0, 1.7.0-1.7.2] or uninstalled
 ├─restricted to versions 1.3 by an explicit requirement, leaving only versions 1.3.0
 └─restricted by compatibility requirements with CUDAdrv [c5f51814] to versions: 1.7.1-1.7.2 or uninstalled — no versions left
   └─CUDAdrv [c5f51814] log:
     ├─possible versions are: [0.8.0-0.8.6, 0.9.0, 1.0.0-1.0.1, 2.0.0, 3.0.0-3.0.1, 3.1.0, 4.0.0-4.0.4, 5.0.0-5.0.1, 5.1.0, 6.0.0] or uninstalled
     └─restricted to versions 6.0.0 by an explicit requirement, leaving only versions 6.0.0

In [ ]:
# From
#    https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
# Classifies MNIST digits with a convolutional network.
# Writes out saved model to the file "mnist_conv.bson".
# Demonstrates basic model construction, training, saving,
# conditional early-exit, and learning rate scheduling.
#
# This model, while simple, should hit around 99% test
# accuracy after training for approximately 20 epochs.


using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON

# Load labels and images from Flux.Data.MNIST
@info("Loading data set")
train_labels = MNIST.labels()
train_imgs = MNIST.images()

# Bundle images together with labels and group into minibatchess
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end

# batch_size = 128 (original) will cause out of memory error. Trying out
# smaller batchsizes.
batch_size = 32
mb_idxs = partition(1:length(train_imgs), batch_size)
train_set = [make_minibatch(train_imgs, train_labels, i) for i in mb_idxs]

# Prepare test set as one giant minibatch:
test_imgs = MNIST.images(:test)
test_labels = MNIST.labels(:test)
test_set = make_minibatch(test_imgs, test_labels, 1:length(test_imgs))

# Define our model.  We will use a simple convolutional architecture with
# three iterations of Conv -> ReLU -> MaxPool, followed by a final Dense
# layer that feeds into a softmax probability output.
@info("Constructing model...")
model = Chain(
    # First convolution, operating upon a 28x28 image
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one, at this point it should be (3, 3, 32, N)
    # which is where we get the 288 in the `Dense` layer below:
    x -> reshape(x, :, size(x, 4)),
    Dense(288, 10),

    # Finally, softmax to get nice probabilities
    softmax,
)

@info("Loading model...")
# Load model and datasets onto GPU, if enabled
# .... unfortunateley neither the current nor the commented-out
#      code will actually make the GUP do anything.
train_set =  train_set |> gpu # gpu.(train_set)
test_set = test_set |> gpu # gpu.(test_set)
model =  model |> gpu #gpu(model)

@info("Precompiling model...")
# Make sure our model is nicely precompiled before starting our training loop
model(train_set[1][1])

@info("Done precompiling model...")


# `loss()` calculates the crossentropy loss between our prediction `y_hat`
# (calculated from `model(x)`) and the ground truth `y`.  We augment the data
# a bit, adding gaussian random noise to our image to make it more robust.
function loss(x, y)
    # We augment `x` a little bit here, adding in random noise
    x_aug = x .+ 0.1f0*gpu(randn(eltype(x), size(x)))

    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

# Train our model with the given training set using the ADAM optimizer and
# printing out performance against the test set as we go.
opt = ADAM(0.001)

@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch_idx in 1:100
    
    @info(@sprintf("[%d]:Starting epoch", epoch_idx))
    
    global best_acc, last_improvement
    # Train for a single epoch
    Flux.train!(loss, params(model), train_set, opt)

    # Calculate accuracy:
    acc = accuracy(test_set...)
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch_idx, acc))

    # If our accuracy is good enough, quit out.
    if acc >= 0.999
        @info(" -> Early-exiting: We reached our target accuracy of 99.9%")
        break
    end

    # If this is the best accuracy we've seen so far, save the model out
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to mnist_conv.bson")
        BSON.@save joinpath(dirname(@__FILE__), "mnist_conv.bson") model epoch_idx acc
        best_acc = acc
        last_improvement = epoch_idx
    end

    # If we haven't seen improvement in 5 epochs, drop our learning rate:
    if epoch_idx - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")

        # After dropping learning rate, give it a few epochs to improve
        last_improvement = epoch_idx
    end

    if epoch_idx - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Loading data set
└ @ Main In[3]:18
┌ Info: Constructing model...
└ @ Main In[3]:46
┌ Info: Loading model...
└ @ Main In[3]:69
┌ Info: Precompiling model...
└ @ Main In[3]:77
┌ Info: Done precompiling model...
└ @ Main In[3]:81
┌ Info: Beginning training loop...
└ @ Main In[3]:100
┌ Info: [1]:Starting epoch
└ @ Main In[3]:105
┌ Info: [1]: Test accuracy: 0.9595
└ @ Main In[3]:113
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[3]:123
┌ Info: [2]:Starting epoch
└ @ Main In[3]:105
